In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import tensorflow as tf
!pip install ipywidgets
!pip install -U accelerate
!pip install -U transformers
!pip install transformers[torch]
!pip install accelerate -U
!pip install sacrebleu
!pip install evaluate
import transformers
import accelerate

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/a6/b9/44623bdb05595481107153182e7f4b9f2ef9d3b674938ad13842054dcbd8/accelerate-0.26.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.24.1
    Uninstalling accelerate-0.24.1:
      Successfully uninstalled accelerate-0.24.1
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/85/f6/c5065913119c41ecad148c34e3a861f719e16b89a522287213698da911fc/transformers-4.37.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 6.2 MB/s eta 0:00:00
  Obtaining dependency information for huggingface-hub<1.0,>=0.19.3 from https://files.pythonhosted.org/packages/28/03/7d3c7153113ec59cfb31e3b8ee773f5f420a0dd7d26d40442542b96675c3/huggingface_hub-0.20.3-py3-none-any.whl.metadata
  Obtaining dependency

In [2]:
data = pd.read_csv("/kaggle/input/hindienglish-corpora/Hindi_English_Truncated_Corpus.csv")
data.drop(columns=["source"],inplace=True)
data.dropna(inplace=True)

In [3]:
data.shape

(127605, 2)

In [4]:
from datasets import Dataset
data = Dataset.from_pandas( data )
data = data.train_test_split(test_size=0.2)
data

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


DatasetDict({
    train: Dataset({
        features: ['english_sentence', 'hindi_sentence', '__index_level_0__'],
        num_rows: 102084
    })
    test: Dataset({
        features: ['english_sentence', 'hindi_sentence', '__index_level_0__'],
        num_rows: 25521
    })
})

In [5]:
data["train"][0]

{'english_sentence': 'How many of you have had the experience',
 'hindi_sentence': 'आप में से कितनों को ऐसा अनुभव हुआ है',
 '__index_level_0__': 120601}

In [6]:
from transformers import AutoTokenizer

model_checkpoint = "rashid996958/Machine_translation_0-5_epochs"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,token='hidden',from_tf=True)

tokenizer_config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [7]:
inputs = tokenizer(data["train"][0]['english_sentence'], text_target=data["train"][0]['hindi_sentence'])
inputs

{'input_ids': [244, 305, 8, 27, 55, 154, 4, 1684, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [118, 11, 12, 25, 13536, 18, 230, 2538, 206, 5, 0]}

In [8]:
max_length = 128

def preprocess_function(data):
    inputs = [ex for ex in data["english_sentence"]]
    targets = [ex for ex in data["hindi_sentence"]]
    model_inputs = tokenizer( inputs, text_target=targets, max_length=max_length, truncation=True )
    return model_inputs

In [9]:
tokenized_datasets = data.map(preprocess_function,batched=True, remove_columns=data["train"].column_names)

  0%|          | 0/103 [00:00<?, ?ba/s]

  0%|          | 0/26 [00:00<?, ?ba/s]

In [10]:
from transformers import TFAutoModelForSeq2SeqLM
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at rashid996958/Machine_translation_0-5_epochs.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

In [11]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [12]:
import numpy as np
np.object = object   
np.object()

In [13]:
batch_size = 16
tf_train_dataset = model.prepare_tf_dataset( tokenized_datasets["train"], collate_fn=data_collator, shuffle=True, batch_size=batch_size )
tf_eval_dataset = model.prepare_tf_dataset( tokenized_datasets["test"], collate_fn=data_collator, shuffle=False, batch_size=batch_size )

In [14]:
from nltk.translate.bleu_score import corpus_bleu

# Example BLEU calculation function
def compute_bleu_score(predictions, references):
    return corpus_bleu(references, predictions)

@tf.function(jit_compile=True)
def generate_with_xla(batch):
    return model.generate( input_ids=batch["input_ids"], attention_mask=batch["attention_mask"], max_new_tokens=128 )

def compute_metrics():
    all_preds = []
    all_labels = []
    for batch, labels in tqdm(tf_eval_dataset):
        predictions = generate_with_xla(batch)
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        labels = labels.numpy()
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_preds = [pred.strip() for pred in decoded_preds]
        decoded_labels = [[label.strip()] for label in decoded_labels]
        all_preds.extend(decoded_preds)
        all_labels.extend(decoded_labels)
        
    result = compute_bleu_score(all_preds, all_labels)
    return {"bleu": result}

In [15]:
from huggingface_hub import notebook_login, login
# notebook_login()
login(token="hidden", write_permission=True,add_to_git_credential=True)

Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Model Training

In [16]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf
num_epochs = 5

num_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(init_lr=5e-4, num_warmup_steps=0, num_train_steps=num_epochs, weight_decay_rate=0.01 )
model.compile(optimizer=optimizer)

In [17]:
from transformers.keras_callbacks import PushToHubCallback


push_to_hub_callback = PushToHubCallback( output_dir="Machine_translation_5-10_epochs", tokenizer=tokenizer )

model.fit( tf_train_dataset, validation_data=tf_eval_dataset, callbacks=[push_to_hub_callback], epochs=num_epochs )


/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/rashid996958/Machine_translation_5-10_epochs into local empty directory.


Epoch 1/5
6380/6380 [==============================] - ETA: 0s - loss: 4.7036

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}
Adding files tracked by Git LFS: ['source.spm', 'target.spm']. This may take a bit of time if the files are large.


6380/6380 [==============================] - 1851s 285ms/step - loss: 4.7036 - val_loss: 4.5561
Epoch 2/5
6380/6380 [==============================] - ETA: 0s - loss: 4.7027

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}


...

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}


6380/6380 [==============================] - 1811s 284ms/step - loss: 4.7024 - val_loss: 4.5561
Epoch 5/5
6380/6380 [==============================] - ETA: 0s - loss: 4.7027

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}


6380/6380 [==============================] - 1811s 284ms/step - loss: 4.7027 - val_loss: 4.5561


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}


In [19]:
print(compute_metrics()) 

100%|██████████| 1596/1596 [19:32<00:00,  1.36it/s]


{'bleu': 0.3817793153989531}


# Evaluation Pipeline

In [2]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "rashid996958/Machine_translation_5-10_epochs"
translator = pipeline("translation", model=model_checkpoint)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at rashid996958/Machine_translation_5-10_epochs.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [21]:
translator("Hello")

[{'translation_text': 'सलाम'}]

In [23]:
translator("The boy standing next to you")

[{'translation_text': 'लड़का आप के बगल में खड़ा है'}]

In [24]:
print( "-- Training Set -- " )
for i in range(10):
    index = np.random.randint(len(data['train']["english_sentence"]),size=1)
    print("English Sentence: ", data['train']["english_sentence"][index[0]])
    print("Original Hindi Sentence: ", data['train']["hindi_sentence"][index[0]])
    print("Translated Hindi Sentence: ", translator( data['train']["english_sentence"][index[0]])[0]["translation_text"])
    print('\n')

-- Training Set -- 
English Sentence:  In northern Bohemia , Federal Republic of Czechoslovakia , polluted air from the brown-coal belt has caused blight in agricultural areas and has heavily damaged forests .
Original Hindi Sentence:  चेकोस्लोवाकिया गणराज्य के उत्तरी बोहीमिया में भूरी-कोयला क्षेत्र की प्रदूषित वायु से आसपास की फसलें सूख गयी हैं और जंगलों को भी भारी क्षति हुई है .
Translated Hindi Sentence:  उत्तरी बोहवीना गणराज्य में, भूरे रंग की पेटी से हवा के कारण खेती के क्षेत्रों में बहुत ही खराब जंगलों का कारण बन गया है .


English Sentence:  Kaviyo ke Samya Santh Pant (1960)
Original Hindi Sentence:  कवियों के सौम्य संत: पंत (1960)
Translated Hindi Sentence:  कावीयो तायाथथथथथ (1960)


English Sentence:  the population here was 22,000 persons per km.
Original Hindi Sentence:  यहां की जनसंख्या घनत्व २२००० व्यक्ति प्रति वर्ग किलोमीटर था।
Translated Hindi Sentence:  यहाँ की आबादी प्रति मील २,००० लोगों की थी .


English Sentence:  And so the family , hounded from their ancestral home

In [25]:
print( " -- Validation/Testing Set -- " )
for i in range(10):
    index = np.random.randint(len(data['test']["english_sentence"]),size=1)
    print("English Sentence: ", data['test']["english_sentence"][index[0]])
    print("Original Hindi Sentence: ", data['test']["hindi_sentence"][index[0]])
    print("Translated Hindi Sentence: ", translator( data['test']["english_sentence"][index[0]])[0]["translation_text"])
    print('\n')

 -- Validation/Testing Set -- 
English Sentence:  Hadley site of the British Weather Office.
Original Hindi Sentence:  ब्रिटेन के मौसम कार्यालय हैडली केन्द्र की साइट
Translated Hindi Sentence:  ब्रिटिश मौसम विभाग के हैली स्थल.


English Sentence:  Nobody knows where they are.
Original Hindi Sentence:  कोई नहीं जानता कि वे कहाँ हैं.
Translated Hindi Sentence:  कोई नहीं जानता कि वे कहाँ हैं .


English Sentence:  Subhash Chandra Bose (January 23, 1897 - August 18, 1945 controversial) who is also known as Netaji, was a front runner in Indian struggle for independence.
Original Hindi Sentence:  सुभाषचन्द्र बोस (बांग्ला: সুভাষ চন্দ্র বসু शुभाष चॉन्द्रो बोशु) (२३ जनवरी १८९७ - १८ अगस्त १९४५ विवादित) जो नेताजी नाम से भी जाने जाते हैं भारत के स्वतंत्रता संग्राम के अग्रणी नेता थे।
Translated Hindi Sentence:  भारत के स्वतंत्रता के लिए ३, १८९ - अगस्त, १९४५ भी भारतीय संघर्ष में एक सीधा धावक था , जो भी कहा जाता है .


English Sentence:  - is a part of the budget process, only that its presented in t